# 位置编码 (Positional Embedding)

标准 tranformer 架构的自注意力机制本身是“置换不变”的，也就是说无法感知输入序列中 token 的顺序。为了解决这个问题，必须向模型中注入关于 token 位置的信息，这就是位置编码的作用

## 旋转位置编码 (Rotary Position Encoding, RoPE)

- 核心思想：不再将位置信息加在 embedding 中，而是通过数学上的旋转操作在融合位置信息。具体来说，根据 token 的绝对位置，对 Q 和 K 向量在二维子空间中进行旋转
- 关键性质：经过 RoPE 旋转的向量（$Q_m, K_n$，分别在位置$m$和$n$），它们的内积结果中只包含了它们的相对位置 $m - n$，而绝对位置被消除了。使得注意力机制能够天然地关注到相对位置
- 实现方式：
    1. 复数形式：对于位置为 $m$ 的 token，其 query $q$ 经过 RoPE 变换后：
   $$
   f(q, m) = q * e^{i * m * \theta}
   $$
   其中：
   - $\theta = 10000^{-2k/d}$ 是频率参数
   - k 是索引维度，d 是向量维度
   - i 是虚数单位
    2. 矩阵形式
    ```python
    R_m = [[cos(m*theta), -sin(m*theta)],
            [sin(m*theta), cos(m*theta)]]

    # 对相邻两个维度应用RoPE
    [q1, q2] = R_m @ [q1, q2]
    ```

In [9]:
import torch
import typing

def precompute_freqs_cis(dim: int, end: int, theta: float = 10000.0) -> torch.Tensor:
    '''
    Args:
        dim (int): 头的维度（必须为偶数）
        end (int)：序列最大长度
        theta (float)：RoPE 的基数，一般为 10000.0
    Return:
        torch.Tensor: 形状为 (end, dim // 2) 的复数张量
    '''
    # freq for each dimension is 1 / (theta^(2i/dim)), shape: (dim // 2, )
    freq_base = 1.0 / (theta ** (torch.arange(0, dim, 2).float() / dim))

    # create timesteps, shape: (end, )
    t = torch.arange(end, dtype=torch.float32)

    # calculate rotary angle, shape: (end, dim // 2)
    freqs = torch.outer(t, freq_base)

    # 将频率（角度）变成复数形式 cos(freqs) + i*sin(freqs)
    # torch.polar(abs, angle) -> abs * (cos(angle) + i * sin(angle))
    freq_cis = torch.polar(torch.ones_like(freqs), freqs)
    return freq_cis

def apply_rotary_emb(x: torch.Tensor, freqs_cis: torch.Tensor) -> torch.Tensor:
    '''
    Args:
        x (torch.Tensor): input Q or K, shape: (batch, num_heads, seq_len, d_k)
        freq_cis (torch.Tensor): shape: (seq_len, d_model // 2)
    Return:
        torch.Tensor: shape: similar to x
    '''
    # 1. 将 x 的最后一个维度看作 D//2 个复数，x: (..., D) -> (..., D//2, 2)
    x_reshaped = x.float().reshape(*x.shape[: -1], -1, 2)
    # x_complex: (..., D//2)
    x_complex = torch.view_as_complex(x_reshaped)

    # 2. 调整 freqs_cis 的形状以进行广播，(seq_len, D//2) -> (1, 1, seq_len, D // 2)
    freqs_cis = freqs_cis.unsqueeze(0).unsqueeze(1)

    # 3. 执行复数乘法，实现旋转
    # (batch, num_heads, seq_len, D // 2) * (1, 1, seq_len, D // 2)
    x_rotated = x_complex * freqs_cis

    # 4. 转换为实数形式
    # (..., D // 2) -> (..., D // 2, 2)
    x_out_reshaped = torch.view_as_real(x_rotated)
    # (..., D // 2, 2) -> (..., D)
    x_out = x_out_reshaped.flatten(3)

    return x_out.type_as(x)


In [10]:
# test rotary embedding

# input tensor
x = torch.tensor([[[[1.0, 2.0, 3.0, 4.0],
                    [5.0, 6.0, 7.0, 8.0],
                    [9.0, 10.0, 11.0, 12.0]]]])  # shape: (1, 1, 3, 4)
# precompute freqs_cis
freqs_cis = precompute_freqs_cis(dim=4, end=3)
# apply rotary embedding
x_rotated = apply_rotary_emb(x, freqs_cis)
print(x_rotated)

tensor([[[[  1.0000,   2.0000,   3.0000,   4.0000],
          [ -2.3473,   7.4492,   6.9197,   8.0696],
          [-12.8383,   4.0222,  10.7578,  12.2176]]]])
